In [1]:
import torch
from torch import tensor
from utils.matmul import matmul
import math

# Loading Data

In [2]:
#export
from pathlib import Path
from IPython.core.debugger import set_trace
from fastai import datasets
import pickle, gzip, math, torch, matplotlib as mpl
import matplotlib.pyplot as plt
from torch import tensor

MNIST_URL='http://deeplearning.net/data/mnist/mnist.pkl'

def get_data():
    path = datasets.download_data(MNIST_URL, ext='.gz')
    with gzip.open(path, 'rb') as f:
        ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding='latin-1')
    return map(tensor, (x_train,y_train,x_valid,y_valid))

def normalize(x, m, s): return (x-m)/s

In [3]:
x_train,y_train,x_valid,y_valid = get_data()

train_mean,train_std = x_train.mean(),x_train.std()
train_mean,train_std

x_train = normalize(x_train, train_mean, train_std)
# NB: Use training, not validation mean for validation set
x_valid = normalize(x_valid, train_mean, train_std)

train_mean,train_std = x_train.mean(),x_train.std()
train_mean,train_std

n,m = x_train.shape
c = y_train.max()+1
n,m,c

(50000, 784, tensor(10))

# NN 1

In [4]:
class Module():
    def __call__(self, *args):
        self.args = args
        self.out = self.forward(*args)
        return self.out
    
    def forward(self): raise Exception("Not Implemented")
    def backward(self): self.bwd(self.out, *self.args)

In [5]:
class MSE(Module):
    def forward(self, inp, targ): return (inp.squeeze() - targ).pow(2).mean()
    def bwd(self, out, inp, targ): inp.g = 2*(inp.squeeze()-targ).unsqueeze(-1) / targ.shape[0]

In [6]:
class Relu(Module):
    def forward(self, inp): return inp.clamp_min(0.) - 0.5
    def bwd(self, out, inp): inp.g = (inp>0).float() * out.g

In [7]:
class Lin(Module):
    def __init__(self,m,n): 
        # kaiming init
        self.w = torch.randn(m,n)/math.sqrt(2/m)
        self.b = torch.zeros(n)
    
    def forward(self, inp): 
        return matmul(inp,self.w) + self.b
    
    def bwd(self, out, inp):
        inp.g = out.g @ self.w.t()
        self.w.g = torch.einsum("bi,bj->ij", inp, out.g)
        self.b.g = out.g.sum(0)

In [8]:
class Model():
    def __init__(self):
        self.layers = [Lin(784,50), Relu(), Lin(50,1)]
        self.loss = MSE()
        
    def __call__(self, x, targ):
        for l in self.layers: x = l(x)
        return self.loss(x, targ)
    
    def backward(self):
        self.loss.backward()
        for l in reversed(self.layers): l.backward()

In [10]:
model = Model()

In [11]:
loss = model(x_train, y_train.float())

In [12]:
model.backward()